In [1]:
import matplotlib.pyplot as plt
import numpy as np
import helper
import torch.nn as nn
import torchvision.models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import torchvision.utils
import torch
import torch.nn as nn
from torchinfo import summary
from PIL import Image
from torchvision.transforms import ToTensor
from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
from copy import copy
from collections import defaultdict
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
import torchmetrics
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
batch_size=16


data

In [2]:
class CustomDataset(Dataset):
    def __init__(self, image,label):
        self.img=image
        self.label=label
    def __len__(self):
        return len(self.img)

    def __getitem__(self, idx):
        image_tensor=self.img[idx]
        label_tensor=self.label[idx]
        return image_tensor,label_tensor
    
negative_image_path='../../data/1-cycle_10%_초기데이터/classification/negative/*.tiff'
CIS_image_path='../../data/1-cycle_10%_초기데이터/classification/CIS/*.tiff'
positive_image_path='../../data/1-cycle_10%_초기데이터/classification/positive/*.tiff'

negative_image_list=glob(negative_image_path)
CIS_image_list=glob(negative_image_path)
positive_image_list=glob(negative_image_path)

negative_tensor_image=torch.empty(len(negative_image_list),3,512,512)
negative_tensor_label=torch.zeros(len(negative_image_list),3)
CIS_tensor_image=torch.empty(len(CIS_image_list),3,512,512)
CIS_tensor_label=torch.zeros(len(CIS_image_list),3)
positive_tensor_image=torch.empty(len(positive_image_list),3,512,512)
positive_tensor_label=torch.zeros(len(positive_image_list),3)

tf = ToTensor()
for i in range(len(negative_image_list)):
    image=1-tf(Image.open(negative_image_list[i]))/255
    negative_tensor_image[i]=image
    negative_tensor_label[i][0]=1

for i in range(len(CIS_image_list)):
    image=1-tf(Image.open(CIS_image_list[i]))/255
    CIS_tensor_image[i]=image
    CIS_tensor_label[i][1]=1
    
for i in range(len(positive_image_list)):
    image=1-tf(Image.open(positive_image_list[i]))/255
    positive_tensor_image[i]=image
    positive_tensor_label[i][2]=1



train_tensor_image=torch.cat([negative_tensor_image[:40],CIS_tensor_image[:40]], dim=0)
train_tensor_image=torch.cat([train_tensor_image,positive_tensor_image[:40]], dim=0)
train_tensor_label=torch.cat([negative_tensor_label[:40],CIS_tensor_label[:40]], dim=0)
train_tensor_label=torch.cat([train_tensor_label,positive_tensor_label[:40]], dim=0)

test_tensor_image=torch.cat([negative_tensor_image[40:],CIS_tensor_image[40:]], dim=0)
test_tensor_image=torch.cat([test_tensor_image,positive_tensor_image[40:]], dim=0)
test_tensor_label=torch.cat([negative_tensor_label[40:],CIS_tensor_label[40:]], dim=0)
test_tensor_label=torch.cat([test_tensor_label,positive_tensor_label[40:]], dim=0)

train_tensor_image1=copy(train_tensor_image)
train_tensor_label1=copy(train_tensor_label)

test_tensor_image1=copy(test_tensor_image)
test_tensor_label1=copy(test_tensor_label)
for aug in range(8):
    test_tensor_image1=torch.cat([test_tensor_image1,test_tensor_image], dim=0)
    test_tensor_label1=torch.cat([test_tensor_label1,test_tensor_label], dim=0)
    
for aug in range(7):
    train_tensor_image1=torch.cat([train_tensor_image1,train_tensor_image], dim=0)
    train_tensor_label1=torch.cat([train_tensor_label1,train_tensor_label], dim=0)
    
train_dataset=CustomDataset(train_tensor_image1,train_tensor_label1)
test_dataset=CustomDataset(test_tensor_image1 ,test_tensor_label1)
dataset_size = len(test_dataset)
test_size = int(dataset_size * 0.5)
validation_size = dataset_size-test_size
validation_dataset, test_dataset = random_split(test_dataset, [validation_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

model

In [3]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        # BatchNorm에 bias가 포함되어 있으므로, conv2d는 bias=False로 설정합니다.
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion),
        )

        # identity mapping, input과 output의 feature map size, filter 수가 동일한 경우 사용.
        self.shortcut = nn.Sequential()

        self.relu = nn.ReLU()

        # projection mapping using 1x1conv
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x


class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.shortcut = nn.Sequential()

        self.relu = nn.ReLU()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels*BottleNeck.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels*BottleNeck.expansion)
            )
            
    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x
    
class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=3, init_weights=True):
        super().__init__()

        self.in_channels=64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # weights inittialization
        if init_weights:
            self._initialize_weights()

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self,x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        x = self.conv3_x(output)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    # define weight initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def resnet18():
    return ResNet(BasicBlock, [2,2,2,2])

def resnet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

def resnet50():
    return ResNet(BottleNeck, [3,4,6,3])

def resnet101():
    return ResNet(BottleNeck, [3, 4, 23, 3])

def resnet152():
    return ResNet(BottleNeck, [3, 8, 36, 3])

accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=3).to(device)
model = resnet50()
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)
summary(model,(batch_size,3,512,512))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [16, 3]                   --
├─Sequential: 1-1                        [16, 64, 128, 128]        --
│    └─Conv2d: 2-1                       [16, 64, 256, 256]        9,408
│    └─BatchNorm2d: 2-2                  [16, 64, 256, 256]        128
│    └─ReLU: 2-3                         [16, 64, 256, 256]        --
│    └─MaxPool2d: 2-4                    [16, 64, 128, 128]        --
├─Sequential: 1-2                        [16, 256, 128, 128]       --
│    └─BottleNeck: 2-5                   [16, 256, 128, 128]       --
│    │    └─Sequential: 3-1              [16, 256, 128, 128]       58,112
│    │    └─Sequential: 3-2              [16, 256, 128, 128]       16,896
│    │    └─ReLU: 3-3                    [16, 256, 128, 128]       --
│    └─BottleNeck: 2-6                   [16, 256, 128, 128]       --
│    │    └─Sequential: 3-4              [16, 256, 128, 128]       70,400

In [5]:
from torch import tensor


MIN_loss=5000
train_loss_list=[]
val_loss_list=[]
train_acc_list=[]
val_acc_list=[]
soft=nn.Softmax(dim=0)
for epoch in range(10):
    
    train_count=0
    running_loss = 0.0
    acc_loss=0
    train=tqdm(train_dataloader)
    for x, y in train:
        model.train()
        y = y.to(device).float()
        train_count+=1
        x=x.to(device).float()
        optimizer.zero_grad()  # optimizer zero 로 초기화
        predict = model(x).to(device)
        cost = criterion(predict, y) # cost 구함
        acc=accuracy(torch.argmax(soft(predict), 1), torch.argmax(y, 1))
        cost.backward() # cost에 대한 backward 구함
        optimizer.step() 
        running_loss += cost.item()
        acc_loss+=acc
        train.set_description(f"epoch: {epoch+1}/{10} Step: {train_count+1} loss : {running_loss/train_count:.4f} accuracy: {acc_loss/train_count:.4f}")
    train_loss_list.append((running_loss/train_count))
    train_acc_list.append((acc_loss/train_count).cpu().detach().numpy())
#validation
    model.eval()
    val=tqdm(validation_dataloader)
    val_count=0
    val_running_loss=0.0
    val_acc_loss=0
    
    with torch.no_grad():
        for x, y in val:
            y = y.to(device).float()
            val_count+=1
            x=x.to(device).float()
            
            predict = model(x).to(device)
            acc=accuracy(torch.argmax(soft(predict), 1), torch.argmax(y, 1))
            cost = criterion(predict, y)
            val_running_loss+=cost.item()
            val_acc_loss+=acc
            val.set_description(f"Validation epoch: {epoch+1}/{10} Step: {val_count+1} loss : {val_running_loss/val_count:.4f}  accuracy: {val_acc_loss/val_count:.4f}")
        
            val_loss_list.append((val_running_loss/val_count))
            val_acc_list.append((val_acc_loss/val_count).cpu().detach().numpy())
            
target_names = ['정상', '제자리암종', '악성(종양)']
with torch.no_grad():
    count=0
    test_running_loss=0
    acc_loss=0
    predict_list=torch.empty(0,3)
    y_list=torch.empty(0,3)
    for x, y in test_dataloader:
        y = y.to(device).float()
        count+=1
        x=x.to(device).float()
        
        predict = model(x).to(device)
        acc=accuracy(predict, y)
        cost = criterion(predict, y)
        test_running_loss+=cost.item()
        acc_loss+=acc
        predict_list=torch.cat([predict_list,predict.cpu()], dim=0)
        y_list=torch.cat([y_list,y.cpu()], dim=0)
soft=nn.Softmax()
y_pred = torch.argmax(soft(predict_list), 1).cpu().detach().numpy()
y=torch.argmax(y_list, 1)
print(classification_report(y, y_pred, target_names=target_names))


              precision    recall  f1-score   support

          정상       0.32      1.00      0.49        41
       제자리암종       0.00      0.00      0.00        45
      악성(종양)       0.00      0.00      0.00        42

    accuracy                           0.32       128
   macro avg       0.11      0.33      0.16       128
weighted avg       0.10      0.32      0.16       128



/tmp/ipykernel_1361017/1533318966.py:75: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_pred = torch.argmax(soft(predict_list), 1).cpu().detach().numpy()
/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: Undefi